In [2]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import pickle
from PIL import Image
import os

In [4]:
# Face detection model
faceDetectionModel = "C:/Users/nguye/Downloads/models-20241025T083618Z-001/models/res10_300x300_ssd_iter_140000_fp16.caffemodel"
 
# Mô tả kiến trúc
faceDetectionProto ="C:/Users/nguye/Downloads/models-20241025T083618Z-001/models/deploy.prototxt.txt"
 
# Mô hình mô tả khuôn mặt
faceDescriptor ="C:/Users/nguye/Downloads/models-20241025T083618Z-001/models/openface.nn4.small2.v1.t7"
 
# Sử dụng OpenCV DNN đọc mô hìn nhận diện khuôn mặt
dectectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)
 
# Đọc mô hình mô tả khuôn mặt từ file Torch
descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptor)

In [6]:
# Danh sách hình ảnh chuẩn bị
faces = 'D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/FaceImage-20241025T092759Z-001/FaceImage'
os.listdir(faces)

['123456789_LeNhatTung',
 '151901205_BuiKhanhBinh',
 '162000186_NguyenVanHuong',
 '162000223_ChuNguyenTienThanh',
 '162000309_LeHoangQuan',
 '162000339_HuynhAnhDuy',
 '162000651_NguyễnXuânNam',
 '162000670_nguyenmanhthi',
 '162000947_TranThanhDat',
 '162000989_NguyenThanhSang',
 '162001371_leminhtien',
 '162001391_TranTien',
 '1711031462_VanHuuDuc',
 '172100041_HoangHuuDien',
 '172100043_NguyenHuuHa',
 '172100048_NguyenPhamAnhKhoa',
 '172100062_Trần_Nguyễn_Ngọc_Quý',
 '172100095_NguyenTrungNghia',
 '172100103_tranquangthuan',
 '172100123_NguyenTrongAn',
 '172100124_BuiMinhHieu',
 '172100150_nguyensyhien',
 '172100177_DangNguyenThienAn',
 '172100187_LeAnhQuan',
 '172100191_LeMinhKhoi',
 '172100201_NguyenThanhDat',
 '172100207_DuHieuBao',
 '172100251_NguyenHaiDuy',
 '172100257_LuongThanhPhong',
 '172100264_NgoDangDuy',
 '172100280_NguyenHoangQuan',
 '172100304_LiuGiaLuan',
 '172100309_TranCongSang',
 '172100321_NguyenTrongNghia',
 '172100326_DangVuTrangNguyen',
 '172100337_HaDuyPhuong',


In [8]:
# Trích xuất đặc trưng cho một ảnh
duy = 'D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/FaceImage-20241025T092759Z-001/FaceImage/172100251_NguyenHaiDuy/imageAAA_2300.jpg'
img1 = cv2.imread(duy)
# Copy cái ảnh
image = img1.copy()
 
# Lấy chiều cao và chiều rộng của ảnh
h, w = image.shape[:2]
 
# Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)
 
# Thiết lập đầu vào cho mô hình
dectectorModel.setInput(imgBlob)
 
# Thực hiện việc nhận diện khuôn mặt
detections = dectectorModel.forward()
 
# Kiểm tra xem có khuôn mặt nào hay không?
if (len(detections)>0):
  # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
  i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
  confidence = detections[0, 0, i, 2]
 
  # Kiểm tra độ tin cậy có lớn hơn 0.5
  if (confidence>0.5):
      # Tính toán hộp bao quanh khuôn mặt
      box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
      (startX, startY, endX, endY) =box.astype('int')
 
      # Trích xuất vùng ảnh chứa khuôn mặt ra
      roi = image[startY:endY, startX:endX]
      cv2.imshow("Roi", roi)
 
      # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
      faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)
 
      # Thiết lập đầu vào cho mô hình
      descriptorModel.setInput(faceBlob)
 
      # Thực hiện việc trích xuất đặc trưng
      vectors = descriptorModel.forward()
 
      # Print vectors
      print(vectors)
     

[[ 1.54447034e-01  3.71977761e-02 -6.80909082e-02  6.75102696e-02
   7.29250163e-02  2.01577082e-01  2.11687595e-01  1.16756871e-01
  -1.77371979e-01  7.90897831e-02  1.77602470e-02  6.73213974e-02
   1.02842420e-01 -1.69447273e-01  4.45199050e-02 -8.99053439e-02
   4.83674966e-02  2.22941339e-02 -3.93034071e-02  1.05511956e-01
   5.49442507e-02 -1.05865858e-01 -3.20006907e-02  1.79015507e-03
   2.34351736e-02 -9.62370485e-02 -2.01629534e-01 -1.44623578e-01
  -1.61546376e-02  5.24706133e-02 -1.05921552e-02  1.15622602e-01
  -1.68984190e-01  1.00915089e-01 -3.44217606e-02 -8.16745386e-02
  -3.30771841e-02 -6.94335438e-03  4.63703200e-02 -7.30460361e-02
   7.73004889e-02 -9.97976363e-02  1.94499840e-03  3.83454710e-02
  -1.55742466e-01 -1.08724177e-01  5.50907627e-02 -6.51815115e-03
  -8.82965922e-02  8.85017365e-02  1.22610398e-01 -2.39091292e-02
  -1.45438075e-01  1.11236693e-02  2.17252504e-02 -5.96669577e-02
  -8.59794468e-02  6.58739135e-02  1.04759997e-02 -1.06489910e-02
  -2.54378

In [10]:
# Tạo ra một functions để áp dụng cho nhiều ảnh
def myDetect(image_path):
  # Trích xuất đặc trưng cho một ảnh
  img1 = cv2.imread(image_path)
 
  # Copy cái ảnh
  image = img1.copy()
 
  # Lấy chiều cao và chiều rộng của ảnh
  h, w = image.shape[:2]
 
  # Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
  imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)
 
  # Thiết lập đầu vào cho mô hình
  dectectorModel.setInput(imgBlob)
 
  # Thực hiện việc nhận diện khuôn mặt
  detections = dectectorModel.forward()
 
  # Kiểm tra xem có khuôn mặt nào hay không?
  if (len(detections)>0):
    # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
    i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
    confidence = detections[0, 0, i, 2]
 
    # Kiểm tra độ tin cậy có lớn hơn 0.5
    if (confidence>0.5):
        # Tính toán hộp bao quanh khuôn mặt
        box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
        (startX, startY, endX, endY) =box.astype('int')
 
        # Trích xuất vùng ảnh chứa khuôn mặt ra
        roi = image[startY:endY, startX:endX]
        # cv2_imshow(roi)
 
        # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
        faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)
 
        # Thiết lập đầu vào cho mô hình
        descriptorModel.setInput(faceBlob)
 
        # Thực hiện việc trích xuất đặc trưng
        vectors = descriptorModel.forward()
 
        # Print vectors
        return vectors

In [12]:
# Apply for all images
# Khởi tạo một dictionary
data = dict(data=[], label=[])
 
myDir = 'D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/FaceImage-20241025T092759Z-001/FaceImage'
folders = os.listdir(myDir)
 
# Duyệt qua từng thư mục
for folder in folders:
  path = myDir + "/"+folder
  print(path)
 
  files =  os.listdir(path)
  for fileName in files:
    try:
        # Gọi hàm trích xuất đặc trưng
        vector = myDetect(path+'/'+fileName)
 
        # Kiểm tra việc trích xuất đặc trưng thành công:
        if vector is not None:
          #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
          data['data'].append(vector)
          data['label'].append(folder)
          print('Trích xuất đặc trưng thành công')
    except:
      # Bỏ qua nếu gặp lỗi
      pass

D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/FaceImage-20241025T092759Z-001/FaceImage/123456789_LeNhatTung
D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/FaceImage-20241025T092759Z-001/FaceImage/151901205_BuiKhanhBinh
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/FaceImage-20241025T092759Z-001/FaceImage/162000186_NguyenVanHuong
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích x

In [14]:
data.keys()

dict_keys(['data', 'label'])

In [16]:
# Tạo một series từ dictionary
labelSeries = pd.Series(data['label'])
# Đếm số lần xuất hiện của mỗi nhãn
labelCounts = labelSeries.value_counts()
print(labelCounts)
 
# Save data -> file
print('D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img')
fileName = 'D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/data_face_features.pickle'
pickle.dump(data, open(fileName, mode='wb'))
 

1721030566_LeNguyenHoang     150
1721030848_LeQuocKy          135
1721031129_TranAnhHao        130
172100257_LuongThanhPhong    121
1721030367_LePhuongLinh      119
                            ... 
172100304_LiuGiaLuan          10
151901205_BuiKhanhBinh        10
1721030497_HoThienSinh         9
172100103_tranquangthuan       8
1721030802_NguyenNhatTien      5
Name: count, Length: 88, dtype: int64
D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img


In [20]:
import pickle

# Đường dẫn tới tệp pickle
fileName = 'D:/JS Duy/DNTU/nam4/ki1/thigiacmaytinh/img/data_face_features.pickle'

# Tải dữ liệu từ tệp pickle
with open(fileName, mode='rb') as file:
    data_loaded = pickle.load(file)

# In kích thước của dữ liệu
print(f"Kích thước của dữ liệu: {len(data_loaded)}")

# In ra một số phần tử đầu tiên nếu đó là danh sách
if isinstance(data_loaded, list):
    print("Một số phần tử đầu tiên:", data_loaded[:5])
elif isinstance(data_loaded, dict):
    print("Một số khóa trong từ điển:", list(data_loaded.keys())[:5])

Kích thước của dữ liệu: 2
Một số khóa trong từ điển: ['data', 'label']
